In [ ]:
!export TORCH_CUDA_ARCH_LIST="6.1;7.5;8.0"


In [ ]:
!pip install flask-ngrok
!pip install flask pyngrok firebase-admin pillow requests
!npm install -g localtunnel
!pip install cloudinary

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
added 22 packages in 2s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇Collecting cloudinary
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 4.5 MB/s eta 0:00:00


In [ ]:
# don't restart the cell until it finishes
!pip install torch==2.0.1 torchvision==0.15.2 opencv-python==4.7.0.72 diffusers==0.14.0 \
    transformers==4.27.3 accelerate==0.18.0 clean-fid==0.1.35 "torchmetrics[image]==0.11.4" \
    wandb==0.14.0 matplotlib==3.7.1 tqdm xformers
!pip install huggingface_hub==0.16.4
!pip install jax==0.4.14 jaxlib==0.4.14 flax==0.7.2
!pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.0
    Uninstalling huggingface-hub-0.33.0:
      Successfully uninstalled huggingface-hub-0.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.15.2 requires accelerate>=0.21.0, but you have accelerate 0.18.0 which is incompatible.
peft 0.15.2 requires huggingface_hub>=0.25.0, but you have huggingface-hub 0.16.4 which is incompatible.
gradio 5.31.0 requires huggingface-hub>=0.28.1, but you have huggingface-hub 0.16.4 which is incompatible.
gradio-client 1.10.1 requires huggingface-hub>=0.19.3, but you have huggingface-hub 0.16.4 which is incompatible.
sentence-transformers 4.1.0 requires huggingface-hub>=0.20.0, but you have huggingface-hub 0.16.4 which is incompatible.
sentence-tra

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
def check_category(cloth_url):

    category_query = db.collection("categories").where("image_url", "==", cloth_url).limit(1).stream()
    category_doc = next(category_query, None)

    if not category_doc:
        raise ValueError("Category not found for the cloth image")

    category_data = category_doc.to_dict()
    category_name = category_data.get('category_name', "Unknown Category")

    # Ensure the category is valid
    if category_name not in ['Upper', 'Lower', 'Full']:
        raise ValueError(f"Invalid category: {category_name}")

    return category_name


In [ ]:
import os
import cv2
import subprocess
import gdown
import numpy as np

REPO_DIR = os.path.abspath("pytorch-openpose")
print(REPO_DIR)
MODEL_PATH = os.path.join(REPO_DIR, "model", "body_pose_model.pth")
INFERENCE_PATH = os.path.join(REPO_DIR, "inference_keypoints.py")


def setup_openpose_environment():
    print("🔧 Installing system & Python dependencies...")

    # Install system dependencies
    subprocess.run(["apt-get", "update", "-y"], check=True, capture_output=True)
    subprocess.run([
        "apt-get", "install", "-y",
        "build-essential", "libssl-dev", "libffi-dev", "libbz2-dev",
        "libreadline-dev", "libsqlite3-dev", "libncurses5-dev", "libncursesw5-dev",
        "xz-utils", "tk-dev", "liblzma-dev", "python3-openssl", "git",
        "libc6-dev", "libgomp1", "libgcc-s1", "g++"
    ], check=True, capture_output=True)

    subprocess.run(["pip", "install", "-q", "torch", "torchvision", "opencv-python-headless",
                    "matplotlib", "gdown", "tqdm", "scipy", "onnx", "onnxruntime"], check=True)

    if not os.path.exists(REPO_DIR):
        print("📁 Cloning pytorch-openpose repo...")
        subprocess.run([
            "git", "clone", "-q",
            "https://github.com/Hzzone/pytorch-openpose.git"
        ], check=True, capture_output=True)

    os.makedirs(os.path.join(REPO_DIR, "inputs"), exist_ok=True)
    os.makedirs(os.path.join(REPO_DIR, "skeleton_results"), exist_ok=True)
    os.makedirs(os.path.join(REPO_DIR, "keypoints_results"), exist_ok=True)

    # Download model checkpoint
    if not os.path.exists(MODEL_PATH):
        print("⬇ Downloading OpenPose model...")
        os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
        gdown.download(
            "https://drive.google.com/uc?id=1UduPqS9IJdfhMbvQkKybG3ym99Ag788S",
            MODEL_PATH,
            quiet=False
        )

    if not os.path.exists(INFERENCE_PATH):
        print("⬇ Downloading inference script...")
        gdown.download(
            id="1N2B_xindDuLkvFulu5DMJeMYjtunFhLt",
            output=INFERENCE_PATH,
            quiet=False
        )

setup_openpose_environment()

/content/pytorch-openpose
🔧 Installing system & Python dependencies...
📁 Cloning pytorch-openpose repo...
⬇ Downloading OpenPose model...


Downloading...
From (original): https://drive.google.com/uc?id=1UduPqS9IJdfhMbvQkKybG3ym99Ag788S
From (redirected): https://drive.google.com/uc?id=1UduPqS9IJdfhMbvQkKybG3ym99Ag788S&confirm=t&uuid=75b74095-ede4-41c2-9bbe-eb311f84e4dc
To: /content/pytorch-openpose/model/body_pose_model.pth
100%|██████████| 209M/209M [00:03<00:00, 62.6MB/s]


⬇ Downloading inference script...


Downloading...
From (original): https://drive.google.com/uc?id=1N2B_xindDuLkvFulu5DMJeMYjtunFhLt
From (redirected): https://drive.google.com/uc?id=1N2B_xindDuLkvFulu5DMJeMYjtunFhLt&confirm=t&uuid=c97e0496-47a6-4880-991b-13e4d2992675
To: /content/pytorch-openpose/inference_keypoints.py
100%|██████████| 2.91k/2.91k [00:00<00:00, 4.66MB/s]


In [ ]:
import os
import subprocess
import gdown
import cv2

def setup_environment():
    marker_file = ".setup_done"
    if os.path.exists(marker_file):
        print("✅ Environment already set up. Skipping setup.")
        return

    print("⚙️ Setting up environment...")

    # Install system dependencies
    subprocess.run([
        "apt-get", "update", "-y"
    ], check=True)
    subprocess.run([
        "apt-get", "install", "-y",
        "wget", "build-essential", "libssl-dev", "libffi-dev", "libbz2-dev",
        "libreadline-dev", "libsqlite3-dev", "libncurses5-dev", "libncursesw5-dev",
        "xz-utils", "tk-dev", "liblzma-dev", "python3-openssl", "git",
        "libc6-dev", "libgomp1", "libgcc-s1", "g++", "curl", "zlib1g-dev",
        "uuid-dev", "libnss3-dev"
    ], check=True)

    # Download and build Python 3.8
    subprocess.run(["wget", "https://www.python.org/ftp/python/3.8.18/Python-3.8.18.tgz"], check=True)
    subprocess.run(["tar", "-xvf", "Python-3.8.18.tgz"], check=True)
    os.chdir("Python-3.8.18")
    subprocess.run(["./configure", "--enable-optimizations"], check=True)
    subprocess.run(["make", "-j", "2"], check=True)
    subprocess.run(["make", "altinstall"], check=True)
    os.chdir("..")

    # Install pip using ensurepip
    subprocess.run(["/usr/local/bin/python3.8", "-m", "ensurepip"], check=True)
    subprocess.run(["/usr/local/bin/python3.8", "-m", "pip", "install", "--upgrade", "pip", "setuptools", "wheel"], check=True)

    # Install Python dependencies
    subprocess.run([
        "/usr/local/bin/python3.8", "-m", "pip", "install",
        "ninja", "torch", "torchvision", "gdown", "tqdm", "opencv-python-headless"
    ], check=True)

    repo_dir = "Self-Correction-Human-Parsing"
    if not os.path.exists(repo_dir):
        subprocess.run(["git", "clone", "https://github.com/PeikeLi/Self-Correction-Human-Parsing"], check=True)

    os.makedirs(f"{repo_dir}/checkpoints", exist_ok=True)
    os.makedirs(f"{repo_dir}/inputs", exist_ok=True)
    os.makedirs(f"{repo_dir}/outputs", exist_ok=True)

    # Download model checkpoint
    model_path = f"{repo_dir}/checkpoints/final.pth"
    if not os.path.exists(model_path):
        gdown.download(
            "https://drive.google.com/uc?id=1ruJg4lqR_jgQPj-9K0PP-L2vJERYOxLP",
            model_path,
            quiet=False
        )

    with open(marker_file, "w") as f:
        f.write("setup done")

    print("✅ Environment setup completed.")


setup_environment()

⚙️ Setting up environment...


Downloading...
From (original): https://drive.google.com/uc?id=1ruJg4lqR_jgQPj-9K0PP-L2vJERYOxLP
From (redirected): https://drive.google.com/uc?id=1ruJg4lqR_jgQPj-9K0PP-L2vJERYOxLP&confirm=t&uuid=ad4a1edb-fadb-49a4-9bca-00149bfea43c
To: /content/Self-Correction-Human-Parsing/checkpoints/final.pth
100%|██████████| 267M/267M [00:02<00:00, 96.8MB/s]

✅ Environment setup completed.


In [ ]:
def extract_keypoints_skeletons(input_image_path, output_json_path, output_image_path):
    if not os.path.exists(REPO_DIR) or not os.path.exists(MODEL_PATH) or not os.path.exists(INFERENCE_PATH):
        print("⚙ First-time setup required.")
        setup_openpose_environment()
    else:
        print("🔁 Environment already set up. Skipping setup.")

    input_filename = os.path.basename(input_image_path)
    input_path_in_repo = os.path.join(REPO_DIR, "inputs", input_filename)

    img = cv2.imread(input_image_path)
    if img is None:
        raise ValueError(f"Invalid image file: {input_image_path}")
    cv2.imwrite(input_path_in_repo, img)
    print(f"📷 Saved input image to: {input_path_in_repo}")

    # Run inference script
    print("🔍 Running keypoint detection...")
    cmd = (
        f"cd {REPO_DIR} && "
        f"python3 inference_keypoints.py "
        f"--input {input_path_in_repo} "
        f"--output_json keypoints_results "
        f"--output_image skeleton_results "
    )
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

    if result.returncode != 0:
        print("❌ Inference failed!")
        print(result.stderr)
        raise RuntimeError("Keypoint inference script failed.")
    else:
        print("✅ Inference successful.")
        print(result.stdout)

    # Move outputs to desired paths
    base_name = os.path.splitext(input_filename)[0].replace("_0", "")
    generated_json = os.path.join(REPO_DIR, "keypoints_results", f"{base_name}_2.json")
    generated_image_jpg = os.path.join(REPO_DIR, "skeleton_results", f"{base_name}_5.png")

    if not os.path.exists(generated_json):
        raise FileNotFoundError(f"Keypoint JSON not found at {generated_json}")
    if not os.path.exists(generated_image_jpg):
        raise FileNotFoundError(f"Skeleton image not found at {generated_image_jpg}")

    os.makedirs(os.path.dirname(output_json_path), exist_ok=True)
    os.makedirs(os.path.dirname(output_image_path), exist_ok=True)

    # Save JSON
    os.replace(generated_json, output_json_path)
    print(f"📄 Saved keypoints JSON to: {output_json_path}")

    # Save JPG
    os.replace(generated_image_jpg, output_image_path)
    print(f"🖼 Saved skeleton image to: {output_image_path}")


In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage import morphology
from sklearn.cluster import KMeans
import logging

def create_clothing_mask(input_image_path, output_path=None, display_result=True,
                        background_lower_bound=(0, 0, 220), background_upper_bound=(180, 20, 255),
                        morph_kernel_size=3, morph_iterations=1, min_object_size=1000,
                        use_grabcut=True):

    logging.basicConfig(filename='clothing_mask_log.txt', level=logging.INFO,
                        format='%(asctime)s - %(levelname)s - %(message)s')

    try:
        print("⚙ Initializing...")

        if not os.path.exists(input_image_path):
            raise FileNotFoundError(f"Input image not found: {input_image_path}")

        image = cv2.imread(input_image_path)
        if image is None:
            raise ValueError(f"Invalid image file: {input_image_path}")

        print(f"Processing image: {input_image_path}")
        logging.info(f"Processing image: {input_image_path}")

        original_height, original_width = image.shape[:2]
        logging.info(f"Original dimensions: {original_width}x{original_height}")

        max_dim = 1024
        if max(original_height, original_width) > max_dim:
            scale = max_dim / max(original_height, original_width)
            image = cv2.resize(image, (int(original_width * scale), int(original_height * scale)))
            logging.info(f"Resized to: {image.shape[1]}x{image.shape[0]}")

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        inv_gamma = 1.0 / 0.8
        table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
        image_rgb = cv2.LUT(image_rgb, table)

        lab = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=6.0, tileGridSize=(8, 8))
        l = clahe.apply(l)
        lab = cv2.merge((l, a, b))
        image_rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

        image_rgb = cv2.bilateralFilter(image_rgb, d=9, sigmaColor=75, sigmaSpace=75)

        # Additional RGB thresholding for blue channel
        blue_channel = image_rgb[:, :, 2]
        _, blue_mask = cv2.threshold(blue_channel, 150, 255, cv2.THRESH_BINARY_INV)

        hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)
        pixel_values = image_rgb.reshape((-1, 3)).astype(np.float32)
        kmeans = KMeans(n_clusters=5, random_state=42)
        labels = kmeans.fit_predict(pixel_values)
        centers = kmeans.cluster_centers_.astype(np.uint8)
        hsv_centers = cv2.cvtColor(centers.reshape(1, -1, 3), cv2.COLOR_RGB2HSV).reshape(-1, 3)
        background_cluster = np.argmax(hsv_centers[:, 2])
        binary = (labels.reshape(hsv.shape[:2]) != background_cluster).astype(np.uint8) * 255

        # Combine with blue channel mask
        binary = cv2.bitwise_or(binary, blue_mask)

        if background_lower_bound is not None and background_upper_bound is not None:
            color_mask = cv2.inRange(hsv, np.array(background_lower_bound), np.array(background_upper_bound))
            binary = cv2.bitwise_and(binary, cv2.bitwise_not(color_mask))

        # Save intermediate mask for debugging
        logging.info("Saved intermediate mask after K-means and HSV")

        mask = binary // 255
        scale_factor = min(image.shape[0], image.shape[1]) / 1024
        adaptive_kernel_size = max(3, int(morph_kernel_size * scale_factor))
        kernels = [
            cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (adaptive_kernel_size, adaptive_kernel_size)),
            cv2.getStructuringElement(cv2.MORPH_RECT, (adaptive_kernel_size, adaptive_kernel_size)),
            cv2.getStructuringElement(cv2.MORPH_CROSS, (adaptive_kernel_size, adaptive_kernel_size))
        ]

        for i in range(morph_iterations):
            kernel = kernels[i % len(kernels)]
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        min_area = adaptive_kernel_size * adaptive_kernel_size * 4
        mask = morphology.remove_small_objects(mask.astype(bool), min_size=min_area)
        mask = morphology.remove_small_holes(mask, area_threshold=min_area)
        mask = mask.astype(np.uint8) * 255

        mask_float = mask.astype(np.float32) / 255.0
        mask_float = cv2.bilateralFilter(mask_float, d=9, sigmaColor=0.1, sigmaSpace=5)
        binary = (mask_float > 0.5).astype(np.uint8) * 255

        # Save intermediate mask after morphological processing
        logging.info("Saved intermediate mask after morphological processing")


        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, 8, cv2.CV_32S)
        final_mask = np.zeros_like(binary)
        adaptive_min_size = max(min_object_size, int(original_height * original_width * 0.005))
        for i in range(1, num_labels):
            if stats[i, cv2.CC_STAT_AREA] >= adaptive_min_size:
                final_mask[labels == i] = 255

        if max(original_height, original_width) > max_dim:
            final_mask = cv2.resize(final_mask, (original_width, original_height),
                                   interpolation=cv2.INTER_NEAREST)

        if output_path is None:
            output_path = os.path.splitext(input_image_path)[0] + "_mask.png"
        os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)
        cv2.imwrite(output_path, final_mask)
        print(f"✅ Saved mask to: {output_path}")
        logging.info(f"Saved mask to: {output_path}")

        return output_path

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        logging.error(f"Error processing image {input_image_path}: {str(e)}", exc_info=True)
        raise

In [ ]:
import os
import subprocess
import cv2

def human_parsing(input_image_path, output_path=None):
    repo_dir = "Self-Correction-Human-Parsing"
    input_filename = os.path.basename(input_image_path)
    input_image_path_in_repo = f"{repo_dir}/inputs/{input_filename}"

    img = cv2.imread(input_image_path)
    if img is None:
        raise ValueError(f"Invalid image file: {input_image_path}")
    cv2.imwrite(input_image_path_in_repo, img)

    # Run parsing
    print("🔍 Parsing image...")
    cmd = (
        f"cd {repo_dir} && "
        f"/usr/local/bin/python3.8 simple_extractor.py "
        f"--dataset 'atr' "
        f"--model-restore 'checkpoints/final.pth' "
        f"--input-dir 'inputs' "
        f"--output-dir 'outputs'"
    )

    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

    print("----- STDOUT -----\n", result.stdout)
    print("----- STDERR -----\n", result.stderr)

    if result.returncode != 0:
        raise RuntimeError("Parsing failed—see stderr above.")

    expected_output = f"{repo_dir}/outputs/{input_filename.rsplit('.', 1)[0]}.png"
    if not os.path.exists(expected_output):
        raise FileNotFoundError("Expected output not generated")

    if output_path:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        os.replace(expected_output, output_path)
        return output_path

    return expected_output


In [ ]:
!git clone https://github.com/miccunifi/ladi-vton

Cloning into 'ladi-vton'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 109 (delta 25), reused 18 (delta 18), pack-reused 75 (from 1)
Receiving objects: 100% (109/109), 1.54 MiB | 3.91 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [ ]:
from flask import Flask, request, jsonify, send_from_directory
from PIL import Image
from io import BytesIO
import requests
import subprocess
import os
import firebase_admin
from firebase_admin import credentials, firestore, auth
import uuid
from pyngrok import ngrok
import time
from IPython.display import display, HTML
import shutil
from flask_ngrok import run_with_ngrok

# Initialize Firebase
cred = credentials.Certificate("/content/drive/MyDrive/vton-yx1pe4-firebase-adminsdk-fbsvc-15844f65cd.json")
firebase_admin.initialize_app(cred)
db = firestore.client()


In [ ]:
!python3 -m pip install flask-cors


In [ ]:
from flask_cors import CORS
from flask import Flask
from IPython.display import display, HTML

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)

# Keep Colab alive
display(HTML('''
<script>
function keepAlive() {
    console.log("Keeping Colab alive...");
    document.querySelector("colab-toolbar-button#connect").click();
}
setInterval(keepAlive, 60 * 1000);
</script>
'''))

In [ ]:
# Python thread keep-alive
import threading
import torch
import os
import time

# Set environment variables
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # Suppress TensorFlow warnings
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Debugging CUDA errors
os.environ["NVIDIA_TF32_OVERRIDE"] = "0"  # Ensure numerical stability
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Ensure GPU is accessible

def python_keep_alive():
    while True:
        time.sleep(60)
        print("[Keep-Alive] Preventing timeout...")

keep_alive_thread = threading.Thread(target=python_keep_alive, daemon=True)
keep_alive_thread.start()

# Create directories
os.makedirs('input_images', exist_ok=True)
os.makedirs('output', exist_ok=True)
os.makedirs('input_images/upper_body/keypoints', exist_ok=True)
os.makedirs('input_images/upper_body/masks', exist_ok=True)
os.makedirs('input_images/upper_body/label_maps', exist_ok=True)
os.makedirs('input_images/upper_body/skeletons', exist_ok=True)
os.makedirs('input_images/upper_body/images', exist_ok=True)

os.makedirs('input_images/lower_body/keypoints', exist_ok=True)
os.makedirs('input_images/lower_body/masks', exist_ok=True)
os.makedirs('input_images/lower_body/label_maps', exist_ok=True)
os.makedirs('input_images/lower_body/skeletons', exist_ok=True)
os.makedirs('input_images/lower_body/images', exist_ok=True)

os.makedirs('input_images/dresses/keypoints', exist_ok=True)
os.makedirs('input_images/dresses/masks', exist_ok=True)
os.makedirs('input_images/dresses/label_maps', exist_ok=True)
os.makedirs('input_images/dresses/skeletons', exist_ok=True)
os.makedirs('input_images/dresses/images', exist_ok=True)


In [ ]:
import cloudinary
import cloudinary.uploader
import os

# Configure Cloudinary
cloudinary.config(
    cloud_name="dpl6zfv0y",
    api_key="551583568976988",
    api_secret="qWW_j0So54d7g5aW69GRUXHVC9g"
)

def upload_and_get_public_url(image_path):
    # Uploads an image to Cloudinary and returns its public URL

    try:
        response = cloudinary.uploader.upload(image_path)
        cloudinary_url = response['secure_url']
        print(f"Image uploaded to Cloudinary. URL: {cloudinary_url}")
        return cloudinary_url

    except Exception as e:
        print(f"Error processing image: {e}")
        return None


In [ ]:
# Function to create folder and save images, masks, and other data
def add_folder(category_name, person_img, cloth_img):
    if category_name == "Upper":
        unique_id = "1111"
        upper_body_folder = 'input_images/upper_body'
        images_folder = os.path.join(upper_body_folder, "images")
        os.makedirs(images_folder, exist_ok=True)
        cat="upper_body"

        # Define filenames
        person_filename = f"{unique_id}_0.jpg"
        cloth_filename = f"{unique_id}_1.jpg"
        output_filename = f"{unique_id}_0.png"  # Match person image name, different extension

        # Full paths
        person_path = os.path.join(images_folder, person_filename)
        cloth_path = os.path.join(images_folder, cloth_filename)
        output_path = f"/content/output/unpaired/upper_body/{output_filename}"

        # Save images
        person_img.save(person_path)
        cloth_img.save(cloth_path)

        output_labelmask_path = "input_images/upper_body/label_maps/1111_4.png"
        result_path = human_parsing(person_path, output_labelmask_path)

        output_clothmask_path = "input_images/upper_body/masks/1111_1.png"
        create_clothing_mask(cloth_path, output_clothmask_path)

        output_json_path = "input_images/upper_body/keypoints/1111_2.json"
        output_skeletons_path = "input_images/upper_body/skeletons/1111_5.png"
        extract_keypoints_skeletons(person_path, output_json_path, output_skeletons_path)

        # Prepare test pairs file
        test_pairs_path = f"{upper_body_folder}/test_pairs_unpaired.txt"
        with open(test_pairs_path, 'w') as f:
            f.write(f"{os.path.basename(person_path)}    {os.path.basename(cloth_path)}\n")
        shutil.copy(test_pairs_path, "input_images/test_pairs_paired.txt")
        test_path = '/content/input_images/test_pairs_paired.txt'
        return cat,test_pairs_path, person_path, cloth_path, test_path, output_labelmask_path, output_clothmask_path, output_json_path, output_skeletons_path, output_filename

    elif category_name == "Lower":
        unique_id = "1111"
        lower_body_folder = 'input_images/lower_body'
        images_folder = os.path.join(lower_body_folder, "images")
        os.makedirs(images_folder, exist_ok=True)
        cat="lower_body"

        # Define filenames
        person_filename = f"{unique_id}_0.jpg"
        cloth_filename = f"{unique_id}_1.jpg"
        output_filename = f"{unique_id}_0.png"  # Match person image name, different extension

        # Full paths
        person_path = os.path.join(images_folder, person_filename)
        cloth_path = os.path.join(images_folder, cloth_filename)
        output_path = f"/content/output/unpaired/lower_body/{output_filename}"

        # Save images
        person_img.save(person_path)
        cloth_img.save(cloth_path)

        output_labelmask_path = "input_images/lower_body/label_maps/1111_4.png"
        result_path = human_parsing(person_path, output_labelmask_path)

        output_clothmask_path = "input_images/lower_body/masks/1111_1.png"
        create_clothing_mask(cloth_path, output_clothmask_path)

        output_json_path = "input_images/lower_body/keypoints/1111_2.json"
        output_skeletons_path = "input_images/lower_body/skeletons/1111_5.png"
        extract_keypoints_skeletons(person_path, output_json_path, output_skeletons_path)

        # Prepare test pairs file
        test_pairs_path = f"{lower_body_folder}/test_pairs_unpaired.txt"
        with open(test_pairs_path, 'w') as f:
            f.write(f"{os.path.basename(person_path)}    {os.path.basename(cloth_path)}\n")
        shutil.copy(test_pairs_path, "input_images/test_pairs_paired.txt")
        test_path = '/content/input_images/test_pairs_paired.txt'
        return cat,test_pairs_path, person_path, cloth_path, test_path, output_labelmask_path, output_clothmask_path, output_json_path, output_skeletons_path, output_filename

    elif category_name == "Full":
        unique_id = "1111"
        dresses_folder = 'input_images/dresses'
        images_folder = os.path.join(dresses_folder, "images")
        os.makedirs(images_folder, exist_ok=True)
        cat="dresses"

        # Define filenames
        person_filename = f"{unique_id}_0.jpg"
        cloth_filename = f"{unique_id}_1.jpg"
        output_filename = f"{unique_id}_0.png"  # Match person image name, different extension

        # Full paths
        person_path = os.path.join(images_folder, person_filename)
        cloth_path = os.path.join(images_folder, cloth_filename)
        output_path = f"/content/output/unpaired/dresses/{output_filename}"

        # Save images
        person_img.save(person_path)
        cloth_img.save(cloth_path)

        output_labelmask_path = "input_images/dresses/label_maps/1111_4.png"
        result_path = human_parsing(person_path, output_labelmask_path)

        output_clothmask_path = "input_images/dresses/masks/1111_1.png"
        create_clothing_mask(cloth_path, output_clothmask_path)

        output_json_path = "input_images/dresses/keypoints/1111_2.json"
        output_skeletons_path = "input_images/dresses/skeletons/1111_5.png"
        extract_keypoints_skeletons(person_path, output_json_path, output_skeletons_path)

        # Prepare test pairs file
        test_pairs_path = f"{dresses_folder}/test_pairs_unpaired.txt"
        with open(test_pairs_path, 'w') as f:
            f.write(f"{os.path.basename(person_path)}    {os.path.basename(cloth_path)}\n")
        shutil.copy(test_pairs_path, "input_images/test_pairs_paired.txt")
        test_path = '/content/input_images/test_pairs_paired.txt'
        return cat,test_pairs_path, person_path, cloth_path, test_path, output_labelmask_path, output_clothmask_path, output_json_path, output_skeletons_path, output_filename


In [ ]:
import os

# Path to the Python file you want to delete
file_path = '/content/ladi-vton/src/inference.py'

# Delete the file
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"{file_path} has been deleted.")
else:
    print(f"{file_path} does not exist.")

!cp -r /content/drive/MyDrive/inference.py  /content/ladi-vton/src
print("donnne")

/content/ladi-vton/src/inference.py has been deleted.
donnne


In [ ]:
import os

# Path to the Python file you want to delete
file_path = '/content/ladi-vton/src/models/ConvNet_TPS.py'

# Delete the file
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"{file_path} has been deleted.")
else:
    print(f"{file_path} does not exist.")

!cp -r /content/drive/MyDrive/ConvNet_TPS.py  /content/ladi-vton/src/models
print("donnne")

/content/ladi-vton/src/ConvNet_TPS.py does not exist.
donnne


In [ ]:
@app.route('/run_all', methods=['POST'])
def run_all():
    print(f"\n=== New Request Received ===")
    try:
        # checking the content type
        if not request.is_json:
            return jsonify({"error": "Content-Type must be application/json"}), 415

        # get the data and check it
        data = request.get_json()
        user_id = data.get('user_id')
        if not user_id:
            return jsonify({"error": "user_id is required"}), 400

        # find a new request that didn't treat yet
        query = db.collection("TryOn").where("user_id", "==", user_id).where("status", "==", "pending").stream()
        unprocessed_doc = None

        for doc in query:
            user_data = doc.to_dict()
            if user_data.get("status", "") != "done":
                unprocessed_doc = doc
                break

        if not unprocessed_doc:
            return jsonify({"error": "No pending try-on requests for this user"}), 404

        doc = unprocessed_doc
        user_data = doc.to_dict()
        doc_ref = db.collection("TryOn").document(doc.id)

        # check the doc before update
        if not doc_ref.get().exists:
            print(f"❌ Document {doc.id} does not exist. Cannot update.")
            return jsonify({"error": "Document not found for update"}), 404

        doc_ref.update({"status": "processing"})  # update to "processing"
        print(f"✅ Updated request status to 'processing'.")

        person_url = user_data.get('model_img')
        cloth_url = user_data.get('cloth_img')

        if not person_url or not cloth_url:
            return jsonify({"error": "Missing image URLs"}), 400

        # download the image
        try:
            person_response = requests.get(person_url, timeout=10)
            person_response.raise_for_status()
            cloth_response = requests.get(cloth_url, timeout=10)
            cloth_response.raise_for_status()

            person_img = Image.open(BytesIO(person_response.content))
            cloth_img = Image.open(BytesIO(cloth_response.content))
            print(f"✅ Successfully downloaded images.")
        except requests.exceptions.RequestException as e:
            return jsonify({"error": f"Image download failed: {str(e)}"}), 500

        category_name = check_category(cloth_url)
        cat, test_pairs_path, person_path, cloth_path, test_path, output_labelmask_path, output_clothmask_path, output_json_path, output_skeletons_path, output_filename = add_folder(category_name, person_img, cloth_img)
        print("file_Nammmmmmme", output_filename)
        output_path = f"/content/output/unpaired/{cat}/{output_filename}"
        print("file_Nammmmmmme_paaaathhhhh", output_path)
        print("Categooooooooryyyyyyyyyy_Nammmmmmmmmmmmmmmmmmmmmmme", category_name)

        torch.cuda.empty_cache()


        def run_inference():
            print("🔁 Background: Starting inference...")
            try:

                result = subprocess.run([
                  "python", "/content/ladi-vton/src/inference.py",
                    "--dataset", "dresscode",
                    "--dresscode_dataroot", "/content/input_images",
                    "--output_dir", "/content/output",
                    "--test_order", "unpaired",
                    "--category", f"{cat}",
                    "--enable_xformers_memory_efficient_attention",
                    "--use_png",
                    "--compute_metrics",
                    "--batch_size", "1",
                    "--num_workers", "0",
                    "--mixed_precision", "no",

              ], capture_output=True, text=True)

                print(f"✅ Inference stdout:\n{result.stdout}")
                if result.stderr:
                    print(f"⚠️ Inference stderr (non-fatal):\n{result.stderr}")

                if os.path.exists(output_path):
                    final_url = upload_and_get_public_url(output_path)
                    if final_url:
                        print(f"✅ Result URL: {final_url}")
                        doc_ref.update({"output_img": final_url, "status": "done", "category_name": category_name})
                        print("✅ Inference done and result saved.")
                    else:
                        doc_ref.update({"output_img": "error", "status": "upload_failed"})
                        print("❌ Failed to upload output image.")
                else:
                    doc_ref.update({"output_img": "error", "status": "no_output"})
                    print("❌ No output image generated.")

            except subprocess.CalledProcessError as e:
                print(f"❌ Inference failed: {e.stderr}")
                doc_ref.update({"output_img": "error", "status": "inference_failed"})

            finally:
                try:
                    for file_path in [test_pairs_path, person_path, cloth_path, test_path, output_labelmask_path, output_clothmask_path, output_json_path, output_skeletons_path]:
                        if os.path.exists(file_path):
                            os.remove(file_path)
                    print("🧹 Cleaned up temp files.")
                except Exception as cleanup_error:
                    print(f"⚠️ Cleanup error: {cleanup_error}")

        threading.Thread(target=run_inference, daemon=True).start()

        return jsonify({
            "success": True,
            "message": "Processing started",
            "user_id": user_id
        }), 202

    except Exception as e:
        print(f"❌ Server error: {str(e)}")
        return jsonify({"error": "Internal server error", "details": str(e)}), 500


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

!ngrok config add-authtoken 2wKBK5nBSIfiMLhExX5HsOqqfbO_5KHZQY7ZXz2EbAyYDF4td

ngrok.kill()

# use a specific domain
public_url = ngrok.connect(addr=5000, domain="ultimately-stable-lark.ngrok-free.app")
print("Public URL:", public_url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://ultimately-stable-lark.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://ultimately-stable-lark.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
[Keep-Alive] Preventing timeout...

=== New Request Received ===


/usr/local/lib/python3.11/dist-packages/google/cloud/firestore_v1/base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
/tmp/ipython-input-20-823119022.py:16: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  query = db.collection("TryOn").where("user_id", "==", user_id).where("status", "==", "pending").stream()


[Keep-Alive] Preventing timeout...
✅ Updated request status to 'processing'.
✅ Successfully downloaded images.
🔍 Parsing image...
[Keep-Alive] Preventing timeout...
----- STDOUT -----
 Evaluating total class number 18 with ['Background', 'Hat', 'Hair', 'Sunglasses', 'Upper-clothes', 'Skirt', 'Pants', 'Dress', 'Belt', 'Left-shoe', 'Right-shoe', 'Face', 'Left-leg', 'Right-leg', 'Left-arm', 'Right-arm', 'Bag', 'Scarf']

----- STDERR -----
 /usr/local/lib/python3.8/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
simple_extractor.py:106: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github

...

✅ Inference successful.
Saved: /content/pytorch-openpose/skeleton_results/1111_5.png, /content/pytorch-openpose/keypoints_results/1111_2.json

📄 Saved keypoints JSON to: input_images/upper_body/keypoints/1111_2.json
🖼 Saved skeleton image to: input_images/upper_body/skeletons/1111_5.png
file_Nammmmmmme 1111_0.png
file_Nammmmmmme_paaaathhhhh /content/output/unpaired/upper_body/1111_0.png
Categooooooooryyyyyyyyyy_Nammmmmmmmmmmmmmmmmmmmmmme Upper
🔁 Background: Starting inference...
[Keep-Alive] Preventing timeout...
[Keep-Alive] Preventing timeout...
✅ Inference stdout:
initialization method [normal]
initialization method [normal]
compute FID of a folder with dresscode_upper_body statistics
Found 1 images in the folder /content/output/unpaired/upper_body
compute KID of a folder with dresscode_upper_body statistics
Found 1 images in the folder /content/output/unpaired/upper_body

⚠️ Inference stderr (non-fatal):
Jax plugin configuration error: Plugin module %s could not be loaded
Traceback

/usr/local/lib/python3.11/dist-packages/google/cloud/firestore_v1/base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
/tmp/ipython-input-20-823119022.py:16: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  query = db.collection("TryOn").where("user_id", "==", user_id).where("status", "==", "pending").stream()


✅ Updated request status to 'processing'.
✅ Successfully downloaded images.
🔍 Parsing image...
----- STDOUT -----
 Evaluating total class number 18 with ['Background', 'Hat', 'Hair', 'Sunglasses', 'Upper-clothes', 'Skirt', 'Pants', 'Dress', 'Belt', 'Left-shoe', 'Right-shoe', 'Face', 'Left-leg', 'Right-leg', 'Left-arm', 'Right-arm', 'Bag', 'Scarf']

----- STDERR -----
 /usr/local/lib/python3.8/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
simple_extractor.py:106: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more d

INFO:werkzeug:127.0.0.1 - - [28/Jun/2025 17:12:12] "POST /run_all HTTP/1.1" 202 -


✅ Inference successful.
Saved: /content/pytorch-openpose/skeleton_results/1111_5.png, /content/pytorch-openpose/keypoints_results/1111_2.json

📄 Saved keypoints JSON to: input_images/lower_body/keypoints/1111_2.json
🖼 Saved skeleton image to: input_images/lower_body/skeletons/1111_5.png
file_Nammmmmmme 1111_0.png
file_Nammmmmmme_paaaathhhhh /content/output/unpaired/lower_body/1111_0.png
Categooooooooryyyyyyyyyy_Nammmmmmmmmmmmmmmmmmmmmmme Lower
🔁 Background: Starting inference...
[Keep-Alive] Preventing timeout...
[Keep-Alive] Preventing timeout...
✅ Inference stdout:
initialization method [normal]
initialization method [normal]
compute FID of a folder with dresscode_lower_body statistics
Found 1 images in the folder /content/output/unpaired/lower_body
compute KID of a folder with dresscode_lower_body statistics
Found 1 images in the folder /content/output/unpaired/lower_body

⚠️ Inference stderr (non-fatal):
Jax plugin configuration error: Plugin module %s could not be loaded
Traceback